In [1]:
import pandas as pd
import xarray as xr
import numpy as np

In [9]:
ssrd_ds = xr.open_dataset(f'../data/CAMS/cams_forecast/cams_forecast_ssrd_2023.grib',engine='cfgrib')
temp_ds = xr.open_dataset(f'../data/CAMS/cams_forecast/cams_forecast_t2m_2023.grib',engine='cfgrib')
u10_ds = xr.open_dataset('../data/CAMS/cams_forecast/cams_forecast_u10_2023.grib',engine='cfgrib')
v10_ds = xr.open_dataset('../data/CAMS/cams_forecast/cams_forecast_v10_2023.grib',engine='cfgrib')


site_df = pd.read_csv('../data/DeDe_49stations/DeDe_49sites_metadata.csv')

In [ ]:
st = '2023-01-01'
en = '2023-12-31'
time = pd.date_range(st, en, freq='1D')

for i, row in site_df.iterrows():

    print(f'Processing site {row["Site_id"]}...')
    forecast_df = pd.DataFrame()
    for date in time:
        df = ssrd_ds.sel(time=f'{str(date.date())}',latitude=row['lat'],longitude=row['lng'],method='nearest').to_dataframe()
        df['ssrd'] = (df['ssrd'] - df['ssrd'].shift(1).fillna(0)) / 3600
        df['ssrdc'] = (df['ssrdc'] - df['ssrdc'].shift(1).fillna(0)) / 3600
        forecast_df = pd.concat([forecast_df, df], axis=0)

    forecast_df.reset_index(drop=True,inplace=True)
    forecast_df.drop(columns=['time','number','surface','latitude', 'longitude'], inplace=True)
    forecast_df.rename(columns={'valid_time': 'Datetime'}, inplace=True)
    forecast_df.set_index('Datetime', inplace=True)
    forecast_df = forecast_df.tz_localize('UTC').tz_convert('Asia/Bangkok').tz_localize(None)
    forecast_df.reset_index(inplace=True)
    forecast_df.to_csv(f'../data/CAMS/cams_forecast/all_site_1hr/cams_forecast_1hr_2023_site{row['Site_id']:02d}.csv', index=False)

Processing site 49...


In [ ]:
st = '2023-01-01'
en = '2023-12-31'
time = pd.date_range(st, en, freq='1D')

for i, row in site_df.iterrows():
    print(f'Processing site {row["Site_id"]}...')
    forecast_df = pd.DataFrame()
    for date in time:
        df = ssrd_ds.sel(time=f'{str(date.date())}',latitude=row['lat'],longitude=row['lng'],method='nearest').to_dataframe()
        df['ssrd'] = (df['ssrd'] - df['ssrd'].shift(1).fillna(0)) / 3600
        df['ssrdc'] = (df['ssrdc'] - df['ssrdc'].shift(1).fillna(0)) / 3600
        df.set_index('valid_time', inplace=True)
        df = df.resample('15min').asfreq()
        df['ssrd'] = df['ssrd'].interpolate(method='linear')
        df['ssrdc'] = df['ssrdc'].interpolate(method='linear')
        df.reset_index(inplace=True)

        forecast_df = pd.concat([forecast_df, df], axis=0)

    forecast_df.reset_index(drop=True,inplace=True)
    forecast_df.drop(columns=['time','number','surface','latitude', 'longitude'], inplace=True)
    forecast_df.rename(columns={'valid_time': 'Datetime'}, inplace=True)
    forecast_df.set_index('Datetime', inplace=True)
    forecast_df = forecast_df.tz_localize('UTC').tz_convert('Asia/Bangkok').tz_localize(None)
    forecast_df.reset_index(inplace=True)
    forecast_df.to_csv(f'../data/CAMS/cams_forecast/all_site_15min/cams_forecast_15min_2023_site{row['Site_id']:02d}.csv', index=False)

Processing site 49...


In [ ]:
for i, row in site_df.iterrows():
    print(f'Processing site {row["Site_id"]}...')
    df_1hr = pd.read_csv(f'../data/CAMS/cams_forecast/all_site_1hr/cams_forecast_1hr_2023_site{row["Site_id"]:02d}.csv',parse_dates=['Datetime'])
    df_15min = pd.read_csv(f'../data/CAMS/cams_forecast/all_site_15min/cams_forecast_15min_2023_site{row["Site_id"]:02d}.csv',parse_dates=['Datetime'])

    temp_df = temp_ds.sel(latitude=row['lat'], longitude=row['lng'], method='nearest').to_dataframe()
    temp_df = temp_df.drop(index='0 days 12:00:00', level='step')
    temp_df.rename(columns={'valid_time': 'Datetime'}, inplace=True)
    temp_df.set_index('Datetime', inplace=True)
    temp_df = temp_df.tz_localize('UTC').tz_convert('Asia/Bangkok').tz_localize(None)
    temp_df.reset_index(inplace=True)
    temp_df.drop(columns=['number', 'surface', 'latitude', 'longitude'], inplace=True)
    temp_df = temp_df[temp_df['Datetime'].dt.year == 2023]

    temp_15min = temp_df.copy()
    temp_15min.set_index('Datetime', inplace=True)
    temp_15min = temp_15min.resample('15min').asfreq()
    temp_15min['t2m'] = temp_15min['t2m'].interpolate(method='linear')
    temp_15min.reset_index(inplace=True)

    df_1hr = df_1hr.merge(temp_df, on='Datetime', how='right')
    df_1hr.to_csv(f'../data/CAMS/cams_forecast/all_site_1hr/cams_forecast_1hr_2023_site{row['Site_id']:02d}.csv',index=False)

    df_15min = df_15min.merge(temp_15min, on='Datetime', how='right')
    df_15min.to_csv(f'../data/CAMS/cams_forecast/all_site_15min/cams_forecast_15min_2023_site{row['Site_id']:02d}.csv', index=False)


Processing site 49...


In [10]:
for i, row in site_df.iterrows():
    print(f'Processing site {row["Site_id"]}...')
    df_1hr = pd.read_csv(f'../data/CAMS/cams_forecast/all_site_1hr/cams_forecast_1hr_2023_site{row["Site_id"]:02d}.csv',parse_dates=['Datetime'])
    df_15min = pd.read_csv(f'../data/CAMS/cams_forecast/all_site_15min/cams_forecast_15min_2023_site{row["Site_id"]:02d}.csv',parse_dates=['Datetime'])

    u10_df = u10_ds.sel(latitude=row['lat'], longitude=row['lng'], method='nearest').to_dataframe()
    u10_df = u10_df.drop(index='0 days 12:00:00', level='step')
    u10_df.rename(columns={'valid_time': 'Datetime'}, inplace=True)
    u10_df.set_index('Datetime', inplace=True)
    u10_df = u10_df.tz_localize('UTC').tz_convert('Asia/Bangkok').tz_localize(None)
    u10_df.reset_index(inplace=True)
    u10_df.drop(columns=['number', 'surface', 'latitude', 'longitude'], inplace=True)
    u10_df = u10_df[u10_df['Datetime'].dt.year == 2023]

    u10_15min = u10_df.copy()
    u10_15min.set_index('Datetime', inplace=True)
    u10_15min = u10_15min.resample('15min').asfreq()
    u10_15min['u10'] = u10_15min['u10'].interpolate(method='linear')
    u10_15min.reset_index(inplace=True)

    v10_df = v10_ds.sel(latitude=row['lat'], longitude=row['lng'], method='nearest').to_dataframe()
    v10_df = v10_df.drop(index='0 days 12:00:00', level='step')
    v10_df.rename(columns={'valid_time': 'Datetime'}, inplace=True)
    v10_df.set_index('Datetime', inplace=True)
    v10_df = v10_df.tz_localize('UTC').tz_convert('Asia/Bangkok').tz_localize(None)
    v10_df.reset_index(inplace=True)
    v10_df.drop(columns=['number', 'surface', 'latitude', 'longitude'], inplace=True)
    v10_df = v10_df[v10_df['Datetime'].dt.year == 2023]

    v10_15min = v10_df.copy()
    v10_15min.set_index('Datetime', inplace=True)
    v10_15min = v10_15min.resample('15min').asfreq()
    v10_15min['v10'] = v10_15min['v10'].interpolate(method='linear')
    v10_15min.reset_index(inplace=True)

    df_1hr = df_1hr.merge(v10_df, on='Datetime', how='right')
    df_1hr = df_1hr.merge(u10_df, on='Datetime', how='right')
    df_1hr.to_csv(f'../data/CAMS/cams_forecast/all_site_1hr/cams_forecast_1hr_2023_site{row['Site_id']:02d}.csv',index=False)

    df_15min = df_15min.merge(u10_15min, on='Datetime', how='right')
    df_15min = df_15min.merge(v10_15min, on='Datetime', how='right')
    df_15min.to_csv(f'../data/CAMS/cams_forecast/all_site_15min/cams_forecast_15min_2023_site{row['Site_id']:02d}.csv', index=False)

Processing site 1...
Processing site 2...
Processing site 3...
Processing site 4...
Processing site 5...
Processing site 6...
Processing site 7...
Processing site 8...
Processing site 9...
Processing site 10...
Processing site 11...
Processing site 12...
Processing site 13...
Processing site 14...
Processing site 15...
Processing site 16...
Processing site 17...
Processing site 18...
Processing site 19...
Processing site 20...
Processing site 21...
Processing site 22...
Processing site 23...
Processing site 24...
Processing site 25...
Processing site 26...
Processing site 27...
Processing site 28...
Processing site 29...
Processing site 30...
Processing site 31...
Processing site 32...
Processing site 33...
Processing site 34...
Processing site 35...
Processing site 36...
Processing site 37...
Processing site 38...
Processing site 39...
Processing site 40...
Processing site 41...
Processing site 42...
Processing site 43...
Processing site 44...
Processing site 45...
Processing site 46.